In [ ]:
import import_ipynb
import requests
import sentiment
import plotly
import pandas as pd #insertdatatable
from plotly.offline import  init_notebook_mode, plot, iplot
import plotly.express as px

def callSentiment():
    negative_words=sentiment.readNegativeWords()
    positive_words=sentiment.readPositiveWords()
    file=['bas','grab','ktm','ferry','flight','mrt','lrt','taxi']
    article=['bas','grab','ktm','ferry','flight', 'mrt', 'lrt', 'taxi']
    before=[None]*len(file)
    after = [None]*len(file)
    nWords=[None]*len(file)
    pWords=[None]*len(file)
    result=[None]*len(file)
    pPositive=[None]*len(file) #store result persentage
    pNegative=[None]*len(file) #store result percentage
    finalConcl = [None]* len(file)
    count=0
    for file in file:
        text=sentiment.readFile(file) #send file name to read a file
        fullwordlist = sentiment.stripNonAlphaNum(text) #store all words in file in an array
        countWords = sentiment.count(fullwordlist)  #count words before remove stop words
        before[count]=countWords #store count words before remove stop words
        wordlist = sentiment.removeStopwords(fullwordlist, sentiment.stopwords) #remove stopwords
        countWords1 = sentiment.count(wordlist) #count words after remove stop words
        negativeWords=sentiment.calculate(negative_words,wordlist) #bagi negative words and wordlist
        positiveWords=sentiment.calculate(positive_words,wordlist) #bagi positivewords and  wordlist
        print("for : "+article[count]) #just for check
        nWords[count]=negativeWords #store negative words
        pWords[count]=positiveWords #store positive words
        after[count]=countWords1 #store total after remove stop words
        pCount = sentiment.calculatePercentage(positiveWords,countWords1) #calculate percentages for positive
        nCount = sentiment.calculatePercentage(negativeWords,countWords1) #calculate percentages for negative
        conclusion =sentiment.comparePercent(nCount, pCount) #compare the percentages
        result[count]=conclusion #store conclusion in an array
        pPositive[count]=pCount #store percentage positive in an array
        pNegative[count]=nCount #store percentage negatvie in an array
        comparePositiveNegative = sentiment.comparePositiveNegative(pCount, nCount) #calculate score for solution
        finalConcl[count] = comparePositiveNegative #store all score
        count=count+1
    



    df = pd.DataFrame(list(zip(article, before,after,nWords,pWords,pPositive,pNegative,result,finalConcl)),
                  columns = ['article','before','after','negative','positive','%p','%n','conclusion','final'])

    df.to_excel(r'Sentiment.xlsx', index = False)

    print(df)

    fig = px.bar(df, x="article", y=["negative","positive"], title="Frequency Words")
    fig.write_html('countpositivenegative.html', auto_open=True)


    fig = px.scatter(df, x="article", y=["before","after"], title="Frequency Words")
    fig.update_traces(mode="lines+markers+text")

    fig.write_html('countword.html', auto_open=True)